### We  import the  libraries and read our dataset

In [179]:
import pandas as pd
import numpy as np
import matplotlib 
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,median_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from pprint import pprint
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,median_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from pprint import pprint
from sklearn import linear_model
from sklearn.svm import SVR
clf = linear_model.GammaRegressor(alpha=0)
import ast
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.linear_model import RANSACRegressor
pd.set_option('display.max_columns', None)

df = pd.read_csv('listings.csv')


### We remove all the columns that we don't need

In [180]:
df.drop(labels=['host_response_time','host_response_rate','host_acceptance_rate',
        'reviews_per_month','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
        'id','listing_url','scrape_id','last_scraped','name','description','neighborhood_overview',
        'picture_url','host_id', 'host_url','host_name', 'host_since', 'host_location', 'host_about',
        'host_thumbnail_url', 'host_picture_url','host_neighbourhood','neighbourhood','neighbourhood_group_cleansed',
        'latitude','longitude','bathrooms','calendar_updated','calendar_last_scraped','first_review', 'last_review',
        'license','host_total_listings_count','minimum_minimum_nights', 'maximum_minimum_nights',
        'minimum_maximum_nights', 'maximum_maximum_nights','minimum_nights_avg_ntm',
         'availability_60','number_of_reviews_ltm', 'calculated_host_listings_count',
        'availability_90','property_type'],axis=1,inplace=True)


### We transform the price column from 'str' to 'float' 

In [181]:
df['price']=df['price'].str.strip('$')
df['price'] = df['price'].apply(lambda x: float(x.split()[0].replace(',', '')))

### We transform non-numerical labels  to numerical labels

In [182]:

le=LabelEncoder()
df['host_is_superhost']=le.fit_transform(df['host_is_superhost'])
df['host_has_profile_pic']=le.fit_transform(df['host_has_profile_pic'])
df['host_identity_verified']=le.fit_transform(df['host_identity_verified'])
df['has_availability']=le.fit_transform(df['has_availability'])
df['instant_bookable']=le.fit_transform(df['instant_bookable'])
df['neighbourhood_cleansed']=le.fit_transform(df['neighbourhood_cleansed'])

In [183]:
# 'Entire home/apt'-3, 'Hotel room'-2, 'Private room'-1, 'Shared room'-0
df['room_type'] = df['room_type'].map(lambda x: 0 if x=='Shared room' else (1 if x=='Private room' else(2 if x=='Entire home/apt' else 3)))
np.unique(np.array(df['room_type'].astype('str')),return_counts=True)


(array(['0', '1', '2', '3'], dtype=object),
 array([  90,  891, 8463,  138], dtype=int64))

In [184]:
cat=np.unique(np.array(df['bathrooms_text'].astype('str')))[0:-1]
cat[-1]='0.3 '+cat[-1]
cat[-2]='0.5 '+cat[-2]
dict={}

for i,elem in enumerate(cat):
    if 'shared' in elem:
        dict[elem]=float(''.join(i for i in elem if (i.isdigit() or i == ".")))-0.2
    elif 'private' in elem:
        dict[elem]=float(''.join(i for i in elem if (i.isdigit() or i == ".")))-0.1
    else:
        dict[elem]=float(''.join(i for i in elem if (i.isdigit() or i == ".")))
    
a = sorted(dict.items(), key=lambda x: x[1])
j=0
for i in a:
    dict[i[0]]=j
    j+=1

dict['0 baths']=0
dict['Half-bath'] = dict['0.5 Half-bath']
dict['Shared half-bath']=dict['0.3 Shared half-bath']
del dict['0.3 Shared half-bath']
del dict['0.5 Half-bath']
dict['nan']=np.NaN
print(dict)


{'0 baths': 0, '0 shared baths': 0, '1 bath': 6, '1 private bath': 5, '1 shared bath': 4, '1.5 baths': 8, '1.5 shared baths': 7, '12 baths': 27, '2 baths': 10, '2 shared baths': 9, '2.5 baths': 12, '2.5 shared baths': 11, '3 baths': 14, '3 shared baths': 13, '3.5 baths': 16, '3.5 shared baths': 15, '4 baths': 18, '4 shared baths': 17, '4.5 baths': 19, '5 baths': 21, '5 shared baths': 20, '5.5 baths': 22, '6 baths': 23, '6.5 baths': 24, '7 baths': 25, '9.5 baths': 26, 'Half-bath': 3, 'Shared half-bath': 2, 'nan': nan}


In [185]:
df['bathrooms_text'].replace(dict,inplace=True)

In [186]:

df['host_verifications'] = df['host_verifications'].apply(lambda x: ast.literal_eval(x))
df['host_verifications'] = df['host_verifications'].fillna(value=np.nan)
for y in range(9582):
    if type(df['host_verifications'][y])==list:
        df.at[y,'host_verifications'] = len(df['host_verifications'][y])

df['amenities'] = df['amenities'].apply(lambda x: ast.literal_eval(x))
for y in range(9582):
    if type(df['amenities'][y])==list:
        df.at[y,'amenities'] = len(df['amenities'][y])



In [187]:
df['amenities'].apply(type).unique()

array([<class 'int'>], dtype=object)

In [188]:
df=df.apply(pd.to_numeric)

### We use one hot encoding to handle the neghbourhood_neansed column

In [189]:
df = pd.get_dummies(df, prefix=['neighbourhood_cleansed'], columns=['neighbourhood_cleansed'])

### We remove the price column and split our dataset into a train set and a test set

In [190]:

df_1=df.copy()
y=df_1['price']
df_1.drop(labels='price',axis=1,inplace=True)
x_train, x_test, y_train, y_test = train_test_split(df_1, y, test_size=0.40,random_state=42)

### We handle nan values remove Ocertain outliers and normalize our tsets

In [191]:
x_train=x_train[y_train<800]
y_train=y_train[y_train<800]
y_train=np.log(y_train)
y_test=np.log(y_test)

In [192]:


imp = IterativeImputer(max_iter=20, random_state=0)
imp.fit(x_train)
x_train= imp.transform(x_train)
x_test= imp.transform(x_test)

In [193]:

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)



### Random Forest Regression

In [194]:

model = RandomForestRegressor()
model.fit(x_train, y_train)
predictions=model.predict(x_test)

y_test_=np.exp(y_test)
predictions_=np.exp(predictions)
print(f'Mean Absolute Percentage Error= {mean_absolute_percentage_error(y_test_, predictions_)}')
print(f'Median Absolute Error= {median_absolute_error(y_test_, predictions_)}')
print(f'Mean Absolute Error= {mean_absolute_error(y_test_, predictions_)}')

Mean Absolute Percentage Error= 0.3090522538548155
Median Absolute Error= 11.475271349724771
Mean Absolute Error= 36.447877109398156


#### Random Hyperparameter Grid

In [195]:
pprint(model.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [196]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]}


#### Random Search training

In [197]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 10, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)

# Fit the random search model
rf_random.fit(x_train, y_train)

In [198]:
# View the best parameters from fitting the random search
rf_random.best_params_

### Random Forest after hyperparameters' tuning

In [199]:

# With the tuned hyperparameters from Random Search 
model = RandomForestRegressor(n_estimators = 400, min_samples_split = 2, min_samples_leaf = 1,
                              max_features = 'log2', max_depth = None, bootstrap = False)
model.fit(x_train, y_train)
predictions=model.predict(x_test)

y_test_=np.exp(y_test)
predictions_=np.exp(predictions)
print(f'Mean Absolute Percentage Error= {mean_absolute_percentage_error(y_test_, predictions_)}')
print(f'Median Absolute Error= {median_absolute_error(y_test_, predictions_)}')
print(f'Mean Absolute Error= {mean_absolute_error(y_test_, predictions_)}')

Mean Absolute Percentage Error= 0.2891002467880968
Median Absolute Error= 10.882625331644931
Mean Absolute Error= 35.55695879949372


### Polynomial Regression

In [200]:
# define the pipeline and train model
model2 = Pipeline([('poly', PolynomialFeatures(degree=1)),
                  ('linear', LinearRegression(fit_intercept=False))])
                  
model2.fit(x_train, y_train)
predictions2=model2.predict(x_test)

y_test_ = np.exp(y_test)
predictions_ = np.exp(predictions2)
print(f'Mean Absolute Percentage Error = {mean_absolute_percentage_error(y_test_, predictions_)}')
print(f'Median Absolute Error = {median_absolute_error(y_test_, predictions_)}')
print(f'Mean Absolute Error = {mean_absolute_error(y_test_, predictions_)}')

Mean Absolute Percentage Error = 0.37612652333486596
Median Absolute Error = 13.811843341454555
Mean Absolute Error = 40.48833140334374


### Robust Regression — RANSAC

In [201]:

# Set RANSAC hyperparameters
ransac = RANSACRegressor(LinearRegression(),
		max_trials=50, 		# Number of Iterations
		min_samples=20, 		# Minimum size of the sample
		loss='absolute_error', 	# Metrics for loss
		residual_threshold=15 	# Threshold
		)

# Train model
ransac.fit(x_train, y_train)
predictions3=ransac.predict(x_test)

y_test_ = np.exp(y_test)
predictions_ = np.exp(predictions3)
print(f'Mean Absolute Percentage Error = {mean_absolute_percentage_error(y_test_, predictions_)}')
print(f'Median Absolute Error = {median_absolute_error(y_test_, predictions_)}')
print(f'Mean Absolute Error = {mean_absolute_error(y_test_, predictions_)}')

Mean Absolute Percentage Error = 0.37613265631658094
Median Absolute Error = 13.814462103399997
Mean Absolute Error = 40.48852482534842


### GLM: Gamma link function

In [202]:

clf.fit(x_train, y_train)
predictions4=clf.predict(x_test)

y_test_ = np.exp(y_test)
predictions_ = np.exp(predictions4)
print(f'Mean Absolute Percentage Error = {mean_absolute_percentage_error(y_test_, predictions_)}')
print(f'Median Absolute Error = {median_absolute_error(y_test_, predictions_)}')
print(f'Mean Absolute Error = {mean_absolute_error(y_test_, predictions_)}')

Mean Absolute Percentage Error = 0.3789044515241129
Median Absolute Error = 13.64477764536581
Mean Absolute Error = 41.487611527964866


### Support Vector Regression

In [203]:

# Choose regression method and set hyperparameter
svr_rbf = SVR(C=1.0, epsilon=0.2, kernel='rbf')

# Training of the regression model
svr_rbf.fit(x_train, y_train)
y_pred = svr_rbf.predict(x_test)

y_test_ = np.exp(y_test)
predictions_ = np.exp(y_pred)

print(f'Mean Absolute Percentage Error = {mean_absolute_percentage_error(y_test_, predictions_)}')
print(f'Median Absolute Error = {median_absolute_error(y_test_, predictions_)}')
print(f'Mean Absolute Error = {mean_absolute_error(y_test_, predictions_)}')

Mean Absolute Percentage Error = 0.3351595356152841
Median Absolute Error = 11.934978401631845
Mean Absolute Error = 38.785403463786245
